# AI Assistant
## Group Members:
 - Krylova Alena
 - Dudic Mateja
 - Saavedra Triana Erwin Omar
 - Maringer Kelvin

## Python Version:
 - 3.13

## Contributions:
 - Krylova Alena:
     - ...
 - Dudic Mateja:
     - ...
 - Saavedra Triana Erwin Omar:
     - ...
 - Maringer Kelvin:
     - ...

# FIRST TIME SETUP
# ----------------
# MAKE SURE THAT THIS CELL RUNS WITHOUT ERRORS BEFORE PROCEEDING
# ----------------

In [ ]:
# All the packages that have to be installed should be listed here
%pip install numpy pandas matplotlib seaborn kagglehub ipywidgets --quiet
# This will filter out the output from Jupyter Notebooks when committing to git, so that diffs are cleaner
! git config filter.strip-notebook-output.clean 'jupyter nbconvert --ClearOutputPreprocessor.enabled=True --to=notebook --stdin --stdout --log-level=ERROR'

import kagglehub
import platform

# Download latest version
dataset_path = kagglehub.dataset_download("prince7489/daily-ai-assistant-usage-behavior-dataset") + ("/Daily_AI_Assistant_Usage_Behavior_Dataset.csv" if platform.system() != "Windows" else "\\Daily_AI_Assistant_Usage_Behavior_Dataset.csv")

print("Path to dataset files:", dataset_path)

# ----------------------
# ----------------------

In [ ]:
#All the imports should be listed here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sea


## *Dataset Overview*
 - Additional Notes etc...


In [ ]:
## Code for the dataset overview Here

## *Data Quality Check*
 - Additional Notes etc...


In [ ]:
## Code for the data quality check Here

## *Data-Preprocessing*
 - Additional Notes etc...

In [ ]:
# From here you can take for the Data Quality Check
RAW_data = pd.read_csv(dataset_path)

# first we check the number of missing values in each column
print(RAW_data.isnull().sum())
# after checking we can see that there are no missing values in the dataset

# so for the outliers in this dataset , I think the best aproach would be to leave them and just mark them as outliers. In this dataset the outliers might be relevant data from users that have a diferent behavior than the average user, so removing them would mean losing relevant data.

# now we will check for outliers using the IQR method

y = RAW_data.select_dtypes(include=[np.number])
print(y)
for column in y:
    quartile_min = RAW_data[column].quantile(0.25)
    quartile_max = RAW_data[column].quantile(0.75)

    IQR = quartile_max - quartile_min

    lower_bound = quartile_min - 1.5 * IQR
    upper_bound = quartile_max + 1.5 * IQR

    outliers_promt_length = RAW_data[(RAW_data[column] < lower_bound) | (RAW_data[column] > upper_bound)].count()
    outliers_promt_length = outliers_promt_length.sum()

    print(f"the number of outliers in {column} is the following: \n{outliers_promt_length}")

 # Start of the data preprocessing

# after checking we can see that there are no outliers in the dataset, surprinsing but good.
# next we will check for duplicates in the dataset
x = RAW_data.duplicated().sum()
print(f"the number of duplicates in the dataset is the following: \n{x}")
# after checking we can see that there are no duplicates in the dataset

#

# so we continue with creating the required columns


# I made an funtion for this part so its easier to read , and taking noticing that the timestamp is an string i decided to slice the string to get the hour part and then convert it to int to compare it
def timeOfDay(hour):
    if 5 <= hour <= 11:
        return "morning"
    elif 12 <= hour <= 17:
        return "afternoon"
    elif 18 <= hour <= 22:
        return "evening"
    else:
        return "Night"

RAW_data["timeOfDay"] = RAW_data["timestamp"].apply(lambda x:timeOfDay(int(x[11:-6])))
RAW_data["year"] = RAW_data["timestamp"].apply(lambda x:int(x[0:4]))

# now we are going to convert the columns and timestamp to their proper datatypes

RAW_data["timestamp"] = pd.to_datetime(RAW_data["timestamp"])
RAW_data["device"] = RAW_data["device"].astype("category")
RAW_data["assistant_model"] = RAW_data["assistant_model"].astype("category")
RAW_data["timeOfDay"] = RAW_data["timeOfDay"].astype("category")
RAW_data["usage_category"] = RAW_data["usage_category"].astype("category")

# note that the numericals stay the same, and I decided to leave year as a number

RAW_data




## *Data Analysis*
 - Additional Notes etc...

In [ ]:
print("helo")

In [ ]:
## code for data analysis here
print("helo")